In [2]:
import cProfile
import os
import statistics as st
import plotly.graph_objects as go
import random as rd
import time
import pandas as pd
import math
from collections import defaultdict
import numpy as np
import itertools as it
import Utilities.experiment_utils as eu
import unit_test as ut
import Games.mnk as mnk
import Games.Carcassonne.Carcassonne as carc
#import Games.carcassonne_older as csn
import Agents.random as arand
import Agents.vanilla_mcts as mcts 
import Agents.siea_mcts as siea_mcts
import Agents.mcts_rave as rave_mcts
import Agents.one_step_lookahead as osla
import Agents.mcts_solver as mcts_solver
import Games.function_optimisation as fo
import Utilities.logs_management as lm
import Games.chess_64 as chess_64
import chess
import matplotlib.pyplot as plt
import shutil

#cProfile.run("wins =  random_games(10000, base_gs)")
#ut.run()

In [6]:
game_state = chess_64.GameState()
game_state.set_initial_state()
agents = [osla.OSLA_Wins(max_probing_size=1), osla.OSLA_Wins(), arand.RandomPlayer()]
games = 30
final_df = pd.DataFrame()
for agent in agents:
    start_time = time.time()
    fm_calls= []
    game_length = []
    total_decisive = 0
    white_wins = 0
    for game_id in range(games):
        duplicate_state = game_state.duplicate()
        total_fm = 0
        turns = 0
        while not duplicate_state.is_terminal:
            move = agent.choose_action(duplicate_state)
            duplicate_state.make_action(move)
            total_fm += agent.current_fm + 1
            turns += 1
        if duplicate_state.winner is not None:
            total_decisive += 1
            if duplicate_state.winner == 0:
                white_wins += 1
        fm_calls.append(total_fm)
        game_length.append(turns)
    df = pd.DataFrame({"agent": agent.name, "avg_fm_calls": st.mean(fm_calls), "std_fm_calls":st.stdev(fm_calls), "avg_game_length": st.mean(game_length), "std_game_length":st.stdev(game_length), "games": games, "decisive_rate": total_decisive/games, "white_wins":white_wins/games, "avg_game_time": (time.time() - start_time)/games}, index = {0})
    df = pd.concat([df, agent.agent_data()], axis = 1)
    df = pd.concat([df, game_state.logs_data()], axis = 1)
    final_df = pd.concat([final_df, df])
print(final_df)
final_df.to_csv(os.path.join("Outputs","Random_OSLA_comparison_" + str(games) + ".csv"), index = False)

       agent  avg_fm_calls  std_fm_calls  avg_game_length  std_game_length  \
0  OSLA_Wins        4759.4   3017.971397       175.233333       137.360759   
0  OSLA_Wins        3521.1   2471.977016       134.800000       115.977525   
0     Random         349.1    109.266315       349.100000       109.266315   

   games  decisive_rate  white_wins  avg_game_time     Player  ...    45  \
0     30       0.866667         0.4       2.439214  OSLA_Wins  ...  None   
0     30       0.800000         0.3       1.633961  OSLA_Wins  ...  None   
0     30       0.166667         0.1       0.038899     Random  ...  None   

     46    47 player_turn Reward_p0 Reward_p1 Turn Winner Is_terminal  \
0  None  None           0      None      None    1   None       False   
0  None  None           0      None      None    1   None       False   
0  None  None           0      None      None    1   None       False   

  N_available_actions  
0                  20  
0                  20  
0                

In [4]:
#View tree on each iteration - STEP 1

#parameters
game_idx = 6
agent_idx = 3
agent_rollouts = 1
#puzzle_idx = 66 #matein3
puzzle_idx = 13 #matein2
#max_iterations = 0
max_iterations = 5000
iterations_logs_step = 10

#database
games = ["chess", "ttt", "carcassonne", "carcassonne_1", "carcassonne_discard","chess_puzzle", "ttt_setup"]
game = games[game_idx]
print("Game:", game)
agent_types = ["mcts", "rave_mcts", "siea_mcts", "mcts_solver"]
agent_type = agent_types[agent_idx]
print("Agent:", agent_type)

#agent definition
if agent_type == "mcts":
    mcts_agent = mcts.MCTS_Player(max_iterations = max_iterations, rollouts=agent_rollouts)
elif agent_type == "rave_mcts":
    mcts_agent = rave_mcts.MCTS_RAVE(max_iterations = max_iterations, rollouts=agent_rollouts)
elif agent_type == "siea_mcts":
    mcts_agent = siea_mcts.SIEA_MCTS_Player(max_iterations = max_iterations, rollouts=agent_rollouts)
elif agent_type == "mcts_solver":
    mcts_agent = mcts_solver.MCTS_Solver(max_iterations = max_iterations, rollouts=agent_rollouts)

#game definition
if game == "chess":
    game_state = chess_64.GameState()
    game_state.set_initial_state()
elif game == "ttt":
    game_state = mnk.GameState(3,3,3)
    game_state.set_initial_state()
elif game == "ttt_setup":
    game_state = mnk.GameState(3,3,3)
    game_state.set_board(board_items = {(1,1):0, 
                                    (0,0):0, 
                                    (2,2):1, 
                                    (0,1):1,
                                    #(1,0):0
                                    }, player_turn = 0)
elif game == "carcassonne":
    game_state = carc.CarcassonneState()
    game_state.set_initial_state()
elif game == "carcassonne_1":
    game_state = carc.CarcassonneState(initial_tile_quantities=[1 for _ in range(24)])
    game_state.set_initial_state()
elif game == "carcassonne_discard":
    game_state = carc.CarcassonneState(initial_tile_quantities=[1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0])
    game_state.set_initial_state()
elif game == "chess_puzzle":
    game_state = chess_64.GameState()
    #Load lichess dataset
    #lichess_db = pd.read_csv("Datasets/lichess_db_puzzle_subsample.csv")
    lichess_db = pd.read_csv("Datasets/lichess_1000_most_played.csv")
    
    puzzle_row = lichess_db.iloc[puzzle_idx]
    game_state.set_puzzle_lichess_db(puzzle_row)
    print("Move to look for:", puzzle_row["Moves"].split(" ")[1])
    print("Moves to look for:", puzzle_row["Moves"])
    print(puzzle_row["Themes"])

#Initialize mcts agent
if max_iterations == 0:
    random_action = mcts_agent.choose_action(game_state)
    print("N available_moves:", len(game_state.available_actions))
    print("Root node:", mcts_agent.view_mcts_tree())
    game_state.view_game_state()
    print(mcts_agent.player)

Game: ttt_setup
Agent: mcts_solver


In [ ]:
#Puzzle search
by_criteria=True
theme = "mateIn2"
next_puzzle = 2
#puzzle_idx = 10

lichess_db = pd.read_csv("Datasets/lichess_1000_most_played.csv")

puzzles_found = 0
if by_criteria:
    for puzzle_idx, puzzle_row in lichess_db.iterrows():
        if theme in puzzle_row["Themes"]:
            if next_puzzle == puzzles_found:
                print("Puzzle found at index:", puzzle_idx)
                break
            else:
                puzzles_found += 1

puzzle_row = lichess_db.iloc[puzzle_idx]
game_state.set_puzzle_lichess_db(puzzle_row)
print("Move to look for:", puzzle_row["Moves"].split(" ")[1])
print("Moves to look for:", puzzle_row["Moves"])
print(puzzle_row["Themes"])
game_state.board

In [52]:
#View tree on each iteration - STEP 2 - Iterate

#Parameters
run_type = "full_run"

#By iteration
if run_type == "by_iteration":
    iterations = 100
    for it in range(iterations):
        mcts_agent.iteration()

#Full run
if run_type == "full_run":
    mcts_agent.choose_action(game_state)

print("Current decision node: ", mcts_agent.recommendation_policy())
print("Iterations: ", mcts_agent.current_iterations)
print("FM calls: ", mcts_agent.current_fm)
#print(mcts_agent.view_mcts_tree())
print(mcts_agent.view_action_stats())
#print(mcts_agent.view_proven_tree())

Current decision node:  (x1,y0,t0)
Iterations:  27
FM calls:  140

0:Decision_node, edge:None, visits:27, avg_rwd:inf, children:5,from:5
1:Decision_node, edge:(x2,y0,t0), visits:4, avg_rwd:0.5, children:3,from:4, tree_policy_formula:1.78
2:Decision_node, edge:(x1,y2,t0), visits:4, avg_rwd:0.5, children:3,from:4, tree_policy_formula:1.78
3:Decision_node, edge:(x2,y1,t0), visits:5, avg_rwd:0.6, children:4,from:4, tree_policy_formula:1.75
4:Decision_node, edge:(x0,y2,t0), visits:5, avg_rwd:0.6, children:4,from:4, tree_policy_formula:1.75
5:Decision_node, edge:(x1,y0,t0), visits:9, avg_rwd:inf, children:4,from:4, tree_policy_formula:inf


In [53]:
print(mcts_agent.view_proven_tree())


0:Decision_node, edge:None, visits:27, avg_rwd:inf, children:5,from:5, tree_policy:Nonechildren:5
--5:Decision_node, edge:(x1,y0,t0), visits:9, avg_rwd:inf, children:4,from:4, tree_policy:inf
----10:Decision_node, edge:(x2,y1,t1), visits:2, avg_rwd:inf, children:1,from:3, tree_policy:-inf
------27:Decision_node, edge:(x1,y2,t0), visits:1, avg_rwd:inf, children:0,from:0, tree_policy:inf
----15:Decision_node, edge:(x1,y2,t1), visits:2, avg_rwd:inf, children:1,from:3, tree_policy:-inf
------22:Decision_node, edge:(x2,y0,t0), visits:1, avg_rwd:inf, children:0,from:0, tree_policy:inf
----17:Decision_node, edge:(x2,y0,t1), visits:2, avg_rwd:inf, children:1,from:3, tree_policy:-inf
------23:Decision_node, edge:(x1,y2,t0), visits:1, avg_rwd:inf, children:0,from:0, tree_policy:inf
----20:Decision_node, edge:(x0,y2,t1), visits:2, avg_rwd:inf, children:1,from:3, tree_policy:-inf
------24:Decision_node, edge:(x1,y2,t0), visits:1, avg_rwd:inf, children:0,from:0, tree_policy:inf


In [ ]:
#Add number of moves to puzzles
#Add moves depth to puzzles

moves_count_list = []
game_state = chess_64.GameState()
lichess_db = pd.read_csv("Datasets/lichess_db_puzzle.csv")

for puzzle_idx, puzzle_row in lichess_db.iterrows():
    game_state.set_puzzle_lichess_db(puzzle_row)
    moves_count_list.append(len(game_state.available_actions))
    if puzzle_idx % 100000 == 0:
        print(puzzle_idx)

        #save in csv progress
        moves_count_df = pd.DataFrame({"Moves":moves_count_list})
        moves_count_df.to_csv("Datasets/lichess_db_puzzle_moves_temp.csv", index=False)

lichess_db["available_actions"] = moves_count_list
lichess_db.to_csv("Datasets/lichess_db_puzzle_plus.csv", index=False)

In [21]:
#Create lichess db subsets

lichess_db = pd.read_csv("Datasets/lichess_db_puzzle.csv")

#lichess_db sorted by nbplays
lichess_db = lichess_db.dropna()
lichess_db = lichess_db.sort_values(by=['NbPlays'], ascending=False)

#Find the 1000 most popular puzzles
lichess_db_subset = lichess_db.head(1000)

#save subset to csv
dataset_path = "Datasets"
lichess_db_subset.to_csv(os.path.join(dataset_path,'lichess_1000_most_played.csv'))

In [6]:
#Analyse puzzles from db

#Read collective results
#experiment_path = os.path.join("Outputs", "Chess_puzzles")
dataset_path = "Datasets"
file_name = "lichess_db_puzzle_plus"
results_df = pd.read_csv(os.path.join(dataset_path, file_name + ".csv"))

tag_dict = {}
all_tags = []
for row in results_df.iterrows():
    tags = row[1]['Themes'].split(' ')
    for tag in tags:
        if tag not in all_tags:
            all_tags.append(tag)

for tag in all_tags:
    subset_df = results_df[results_df['Themes'].str.contains(tag)]
    tag_count = len(subset_df)
    #correct_subset = subset_df[subset_df['solved_ratio'] == 1]
    #tag_correct_count = len(correct_subset)
    tag_dict[tag] = [tag_count/len(results_df),tag_count]
    #get average_rating
    tag_dict[tag].append(subset_df['Rating'].mean())
    tag_dict[tag].append(subset_df['Rating'].std())
    tag_dict[tag].append(subset_df["available_actions"].mean())
    tag_dict[tag].append(subset_df['available_actions'].std())
    tag_dict[tag].append(subset_df["NbPlays"].mean())
    tag_dict[tag].append(subset_df['NbPlays'].std())
    depth = []
    for index, row in subset_df.iterrows():
        depth.append(len(row['Moves'].split(" ")))
    tag_dict[tag].append(st.mean(depth))
    tag_dict[tag].append(st.stdev(depth))

tag_df = pd.DataFrame.from_dict(tag_dict, orient='index', columns=["appearance_ratio","count","average_rating","std_rating","average_available_moves","std_available_moves","average_nb_plays","std_nb_plays","depth","std_depth"])
#tag_df.sort_values('correct_ratio', ascending=False, inplace=True)
print(tag_df)
tag_df.to_csv(os.path.join(dataset_path, "tags_of_" + file_name + ".csv"))

                   appearance_ratio    count  average_rating  std_rating  \
crushing                   0.451511  1540581     1651.862501  522.979503   
hangingPiece               0.043886   149741     1451.271990  519.163192   
long                       0.246149   839874     1781.336075  471.911401   
middlegame                 0.487523  1663457     1596.763763  507.393405   
advantage                  0.297596  1015416     1672.090906  478.519102   
endgame                    0.454833  1551918     1470.654165  555.752307   
short                      0.572219  1952446     1435.915506  470.999834   
pawnEndgame                0.026855    91630     1806.185856  579.461434   
mate                       0.237620   810773     1130.453559  397.885259   
mateIn2                    0.115771   395016     1139.409849  387.479341   
master                     0.079134   270011     1603.876038  540.279205   
fork                       0.146633   500322     1408.495939  479.796451   
trappedPiece

In [14]:
#Collect chess puzzles results - Already in experiment

running = True
puzzle_idx = 0
iterations_logs_step = 50
run = 0
experiment_path = os.path.join("Outputs", "Chess_puzzles")
collective_results = pd.DataFrame()
while running:
    try:
        move_logs = pd.read_csv(os.path.join(experiment_path, "Puzzle_"+str(puzzle_idx), "results_every_"+ str(iterations_logs_step)+".csv"))
        result_logs = pd.read_csv(os.path.join(experiment_path, "Puzzle_"+str(puzzle_idx), "experiment_data.csv"))

    except:
        running = False
        break    

    run_logs = move_logs[move_logs["exp_run"]==run]
    final_row = run_logs.iloc[-1]
    iteration_as_last_choice = []
    first_action_solution = run_logs["expected_move"][0]
    final_last_unchanged_iteration = None
    if final_row["current_chosen_move"] == first_action_solution:
        correct_run_logs = run_logs[run_logs["current_chosen_move"]==first_action_solution]
        last_unchanged_iteration = correct_run_logs["iterations_executed"].max()
        #print("correct_run_logs_len",str(len(correct_run_logs)), "run_logs_len", str(len(run_logs)))
        for i in reversed(range(len(correct_run_logs))[1:]):
            if abs(correct_run_logs.iloc[i]["iterations_executed"] - correct_run_logs.iloc[i-1]["iterations_executed"]) == iterations_logs_step:
                last_unchanged_iteration = correct_run_logs.iloc[i-1]["iterations_executed"]
            else: break
        iteration_as_last_choice.append(last_unchanged_iteration)
        final_last_unchanged_iteration = st.mean(iteration_as_last_choice)
    
    #Add data to results log
    result_logs["final_last_unchanged_iteration"] = [final_last_unchanged_iteration]
    result_logs["puzzle_idx"] = [puzzle_idx]

    #Insert final row
    final_row_df = pd.DataFrame([final_row.tolist()], columns=final_row.index)

    #find repeated columns
    repeated_columns = []
    for column in final_row_df.columns:
        if column in result_logs.columns:
            repeated_columns.append(column)
    #drop repeated columns
    final_row_df = final_row_df.drop(repeated_columns, axis=1)
    
    result_logs = pd.concat([result_logs, final_row_df], axis=1)

    #Append data to collective results
    collective_results = pd.concat([collective_results, result_logs], axis=0, ignore_index=True)
    
    #End of loop routine
    puzzle_idx += 1

#Save collective results
collective_results.to_csv(os.path.join(experiment_path, "collective_results.csv"))

In [ ]:
#Analyse puzzle results by tag

#Read collective results
experiment_path = os.path.join("Outputs", "Chess_puzzles_results")
collective_results_df = pd.read_csv(os.path.join(experiment_path, "collective_results.csv"))

tag_dict = {}
all_tags = []
for row in collective_results_df.iterrows():
    tags = row[1]['Themes'].split(' ')
    for tag in tags:
        if tag not in all_tags:
            all_tags.append(tag)

for tag in all_tags:
    subset_df = collective_results_df[collective_results_df['Themes'].str.contains(tag)]
    tag_count = len(subset_df)
    correct_subset = subset_df[subset_df['solved_ratio'] == 1]
    tag_correct_count = len(correct_subset)
    tag_dict[tag] = [tag_count/len(collective_results_df),tag_count, tag_correct_count / tag_count]
    #get average_rating
    tag_dict[tag].append(subset_df['Rating'].mean())

tag_df = pd.DataFrame.from_dict(tag_dict, orient='index', columns=["appearance_ratio","count",'correct_ratio',"average_rating"])
tag_df.sort_values('correct_ratio', ascending=False, inplace=True)
print(tag_df)

In [ ]:
#Create color palette

color_palette = [ "#B10909" #red
                ,  "#5B8C5A"#green
                ,"#56638A" #blue-purple
                , "#EC7316" #orange
                ,  "#FC738C" ] #pink
color_palette = ["#5B8C5A"
                ,"#56638A"
                , "#EC7316"]
color_palette[2] = eu.color_rgb_to_hex(eu.darken_color(eu.color_hex_to_rgb(color_palette[2]), 0.4))
color_palette[0] = eu.color_rgb_to_hex(eu.brighten_color(eu.color_hex_to_rgb(color_palette[0]), 0.3))
color_palette[1] = eu.color_rgb_to_hex(eu.brighten_color(eu.color_hex_to_rgb(color_palette[1]), 0.1))

eu.view_color_palette(color_palette)

print("color_palette = {}".format(color_palette))

In [ ]:
#MATCH

mcts_agent = mcts.MCTS_Player(max_iterations = 100)
siea_mcts_agent = siea_mcts.SIEA_MCTS_Player(max_iterations = 100)
#game_state = carc.CarcassonneState(initial_tile_quantities=[1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0])
#game_state = carc.CarcassonneState(initial_tile_quantities=[1 for _ in range(24)])
game_state = carc.CarcassonneState()
#game_state = mnk.GameState(3,3,3)
game_state.set_initial_state()
#random_action = mcts_agent.choose_action(game_state)
#random_action = mcts_agent.choose_action(game_state)
#print("Max score", game_state.max_possible_score)
#print(random_action)
#print(len(game_state.available_actions))
#print(mcts_agent.view_mcts_tree())
gp1, gp2 = eu.play_match([mcts_agent, siea_mcts_agent], game_state, 2, os.path.join("Outputs", "test_match_carc2"), random_seed = 1)

In [5]:
#Match with debugger.py

TotalGames = 10
logs = True
random_seed = 1
if random_seed is not None: 
    rd.seed(random_seed)
    np.random.seed(random_seed)
else: 
    random_seed = rd.randint(0, 2**32)
    print("meh")
    rd.seed(random_seed)
    np.random.seed(random_seed)

game_state = carc.CarcassonneState(initial_tile_quantities=[1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,0,0,0,0,0,0,0])
#game_state = carc.CarcassonneState(initial_tile_quantities=[1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0])
#game_state = mnk.GameState(m=3,n=3,k=3)
game_state.set_initial_state()
agent3 = arand.RandomPlayer()
agent2 = mcts.MCTS_Player(max_iterations=100)
players = [agent2, agent3]
games_count = 0
WinnerCount = {0:0, 1:0, "Draw":0}

all_action_logs = pd.DataFrame()
all_game_logs = pd.DataFrame()

ST = time.time()

for game in range(TotalGames):
    #gs = game_state.duplicate()
    gs = game_state.duplicate()

    #Set logs
    action_logs = pd.DataFrame()
    game_logs = pd.DataFrame()
    if logs:
        for p in players:
            p.logs = True

    #Play game
    safe_count = 0
    while not gs.is_terminal:
        start_time = time.time()
        action = players[gs.player_turn].choose_action(gs)
        #action = rd.choice(gs.available_actions)
        selection_time = time.time() - start_time

        #Update logs
        if logs:
            action_log = players[gs.player_turn].choose_action_logs #Assumes this log is single row
            action_log["game_index"] = games_count
            action_log["selection_time"] = selection_time
            action_log["returned_action"] = str(action)
            action_log["pg_player"] = str(players[gs.player_turn])
            action_logs = pd.concat([action_logs, action_log], ignore_index=True)
            game_logs = pd.concat([game_logs, gs.logs_data()], ignore_index=True)

        #safety check      
        safe_count += 1
        if safe_count > 1000: 
            print("Safe count exceeded")
            print(gs.logs_data())
            print("Last action:" + str(action))
            break
        assert safe_count < 1000, "Safe count exceeded"

        #Make action
        gs.make_action(action)

    if logs:
        #Final logs by action
        final_logs_by_action = pd.concat([action_logs, game_logs], axis=1)


        #Final logs by game
        final_logs_by_game_dict = {}
        for i, player in enumerate(players):
            final_logs_by_game_dict["Player_" + str(i)] = str(player)
        final_logs_by_game_dict["game_random_seed"] = random_seed
        final_logs_by_game_dict["game_index"] = games_count
        final_logs_by_game = pd.DataFrame(final_logs_by_game_dict, index=[0])
        final_logs_by_game = pd.concat([final_logs_by_game, gs.logs_data()], axis=1)
            
    games_count += 1
    if gs.winner is None:
        WinnerCount["Draw"] += 1
    else: WinnerCount[gs.winner] += 1
    print("Game", games_count, "ended, winner:", gs.winner, "scores:", gs.Scores)

    all_action_logs = pd.concat([all_action_logs, final_logs_by_action], ignore_index=True)
    all_game_logs = pd.concat([all_game_logs, final_logs_by_game], ignore_index=True)

ET = time.time()

all_action_logs.to_csv(os.path.join("Outputs","test_carc","by_action.csv"))
all_game_logs.to_csv(os.path.join("Outputs","test_carc","by_game.csv"))
print("Games ended, time:", str(ET - ST))
print("WinnerCount",WinnerCount)

In [2]:
#Evolved formulas analysis, now in experiment

logs_path = os.path.join("Outputs","FO_single_decision_new")
data = pd.read_csv(os.path.join(logs_path, "logs_by_run.csv"))
evolved_formula_data = pd.DataFrame()
for agent in data["Player"].unique():
    if "EA" in agent:
        for f_index in data["Function_index"].unique():
            tdata = data[(data["Player"]==agent) & (data["Function_index"]==f_index)]
            fa_data = eu.evolved_formula_analysis(tdata)
            fa_data["Player"] = [agent]
            fa_data["Function_index"] = [f_index]
            evolved_formula_data = pd.concat([evolved_formula_data, fa_data])
evolved_formula_data.to_csv(os.path.join(logs_path, "evolved_formula_analysis.csv"), mode='a')

In [ ]:
dupe_state = game_state.duplicate()
dupe_state.make_action(rd.choice(dupe_state.available_actions))

In [8]:
for file_name in ["evolution_logs.csv"]:
    experiment_path = os.path.join("Outputs","FO_single_decision_new2")
    file_path_list = lm.find_log_files(file_name, experiment_path)
    lm.combine_logs(experiment_path, file_name, file_path_list)

In [ ]:
#Plot histograms

n_bins = 2**7
experiment_path = os.path.join("Outputs","FO_single_decision_new2")
file_path_list = lm.find_log_files("tree_data.csv", experiment_path)
print(["\n" + f for f in file_path_list])
for function_index in range(5):
  data_list = []
  subplot_titles = []
  file_paths = []

  #Add ea vanilla
  paths_to_remove = []
  for file_path in [f for f in file_path_list if "Function_" + str(function_index) in f]:
      agent_name = file_path.split(os.sep)[-2]
      if "_c" in agent_name:
        data_list.append(pd.read_csv(file_path)) #mod to get interesting runs
        if "1_4142" in agent_name:
           agent_name= agent_name.replace("1_4142", "\u221A\u03052\u0305")
        if "0_5" in agent_name:
           agent_name= agent_name.replace("0_5", "0.5")
        agent_name = agent_name.replace("MCTS_", "MCTS ")
        agent_name = agent_name.replace(" c", " c = ")
        subplot_titles += [agent_name]
        file_paths += [file_path]
        paths_to_remove += [file_path]
  for path in paths_to_remove:
    file_path_list.remove(path)

  paths_to_remove = []
  for file_path in [f for f in file_path_list if "Function_" + str(function_index) in f]:
    agent_name = file_path.split(os.sep)[-2]
    if "EA_" in agent_name and not "SIEA_" in agent_name:
      data_list.append(pd.read_csv(file_path))
      agent_name= agent_name.replace("_its", " ")
      agent_name += " iterations"
      subplot_titles += [agent_name]
      file_paths += [file_path]
  for path in paths_to_remove:
    file_path_list.remove(path)

  for file_path in [f for f in file_path_list if "Function_" + str(function_index) in f]:
    agent_name = file_path.split(os.sep)[-2]
    if "SIEA_" in agent_name:
      data_list.append(pd.read_csv(file_path))  #mod to get interesting runs
      agent_name= agent_name.replace("_its", " ")
      agent_name += " iterations"
      subplot_titles += [agent_name]
      file_paths += [file_path]
    
  if subplot_titles != [] and data_list != []:
    fop = fo.GameState(function_index=function_index)

    #print(len(data_list))
    """ #mod to get interesting runs
    interesting_runs = [0,10,16,36,41,45,72,73,97]
    interesting_data = []
    interesting_titles = ["Tunnel","Even","Random", "Exploratory", "Opposite", "Emergent 1", "Emergent 2", "Emergent 3", "UCB1 c=0.25"]
    interesting_titles = ["Behaviour type " + str(i) for i in range(len(interesting_runs))]
    for j in range(10):
      new_data_list = []
      subplot_titles = []
      dat = data_list[1]
      for i in range(j*10,(j+1)*10):
        tdat = dat.loc[dat["run"] == i]
        new_data_list = new_data_list + [tdat]
        if i==40:
          my_dat = tdat
        if i in interesting_runs:
          interesting_data += [tdat]
        subplot_titles += ["Run " + str(i)]
      subdata_list = new_data_list
    """

    plot = eu.fo_tree_histogram_average(data_list,  #mod to get interesting runs
                        fop.function, 
                        "F"+str(function_index+1) , #mod to get interesting runs
                        divisions = 3,
                        n_buckets = n_bins, 
                        subplot_titles = [""] + subplot_titles, #mod to get interesting runs
                        max_x_location=fop.max_location[0],
                        y_ref_value=None)
    #plot.write_image(os.path.join(experiment_path, "Interesting_runs.png"))#"F" + str(function_index+1) + "_results"+ str(n_bins) + '.png'))#, width=800, height=1000) 
    plot.write_image(os.path.join(experiment_path, "F" + str(function_index+1) + "_results"+ str(n_bins) + '.png'), width=800, height=1000) 
    plot.show()
    #0,5,

In [1]:
#LUDO

class LudoArrow():
    def __init__(self,  end_index=None, player_house=None):
        self.player_house = player_house #The player whose house this is
        self.end_index = end_index

class LudoSquare():
    def __init__(self, index, is_safe=False, arrow=None, content=None, next_square_index=None, player_house=None, is_victory_path=False):
        self.index = index
        self.is_safe = is_safe
        self.content = content #A list of pieces
        self.next_square_index = next_square_index #The index of the next square
        self.player_house = player_house #The player whose house this sends to
        self.is_victory_path = is_victory_path #Whether this is a victory path square

class LudoPiece():
    def __init__(self, index, player):
        self.index = index
        self.player = player
        
class GameState():
    def __init__(self, n_players, pieces_per_player=4):
        self.players = ["red", "green", "blue", "yellow"]
        self.n_players = n_players
        self.pieces_per_player = pieces_per_player
    def set_initial_state(self):
        self.board = {}
        self.players_houses = {12:"red", 25:"green", 38:"blue", 51:"yellow"}
        self.players_starting_index = {"red":15, "green":28, "blue":41, "yellow":2}
        for i in range(52):
            if i in [2, 10, 15, 23, 28, 36, 41, 49]: #safe squares
                square = LudoSquare(i, is_safe=True)
            elif i in [6, 19, 32, 45]: #squares with immediate arrows
                arrow = LudoArrow(end_index=i+1)
                square = LudoSquare(i, arrow=arrow)
            elif i in [10, 23, 36, 49]: #squares with arrows to the player houses
                arrow = LudoArrow(end_index=53, player_house=i)
                square = LudoSquare(i, arrow=arrow)
            elif i in [13, 26, 39, 52]:
                square = LudoSquare(i, player_house=self.players_houses[i])







In [ ]:
#LORCANA API
import requests
import json
import pandas as pd

names_request = requests.get('https://api.lorcana-api.com/lists/names')
if names_request.status_code == 200:
    response_string = str(names_request.content).split("\\n")
print("Got ", str(len(response_string)), " names from the API")

names = []
card_dataset = []
failed_cards = []
for raw_name in response_string:
    card_data = None
    attempts = 0
    while attempts < 5:
        if "," in raw_name: #Filtering for missing cards
            name = raw_name.replace("\"", "")
            name = name.replace("\\", "")
            name = name.replace(" ", "")
            name = name.replace(",", "")

            full_string = 'https://api.lorcana-api.com/strict/' + name
            print("Trying string: ", full_string)
            #print(full_string)
            card_data = requests.get(full_string)
            if card_data.status_code == 200:
                print(card_data.content)
                break
            else:
                print('Request failed with status code:', card_data.status_code)
                card_data = None
                attempts += 1
        else:
            print("Skipping: ", raw_name)
            break
        
    #Save the data in a dictionary
    if card_data is not None:
        card_dict = json.loads(card_data.content)
        card_dataset.append(card_dict)
    else:
        failed_cards.append(raw_name)
        print("Failed to get data for: ", raw_name)
    
#Merge all the dicionaries in a single dataframe
df = pd.DataFrame()
for card_dict in card_dataset:
    #card_df = pd.DataFrame(card_dict, index = [0])
    card_df = pd.DataFrame()
    for key,value in card_dict.items():
        print(key, value)
        card_df[key] = [value]
    df = pd.concat([df, card_df], axis = 0)
    #Save df to csv
    df.to_csv("card_data.csv", index = False)